In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ensemble")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ensemble


## 본 코드

### docs

In [3]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader('/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1002_noallergy.txt')
docs = loader.load()

In [4]:
docs

[Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1002_noallergy.txt'}, page_content='카테고리 : 버거\n메뉴 이름 : 맥크리스피 스리라차 마요\n제품소개 : 빠삭한 100% 통닭다리살 케이준 패티에스리라차 마요 소스를 더해 매콤 고소한 맛!\n메뉴 개시일 : 2024.09.13 00:00\n메뉴 마감일 : 2024.10.16 23:59\n신제품 여부 : 신제품\n판매시간 : 오전 10시 30분 ~ 새벽 4시\n영양소 정보:\n중량(g): 289g\n중량(ml): -\n열량: 663kcal\n당: 15g\n단백질: 26g\n포화지방: 7g\n나트륨: 1508mg\n카페인: -\n원산지 정보: 닭고기:태국산(경상지역), 브라질산(그 외 지역)\n\n\n카테고리 : 버거\n메뉴 이름 : 베토디 스리라차 마요\n제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!\n메뉴 개시일 : 2024.09.13 00:00\n메뉴 마감일 : 2024.10.16 23:59\n신제품 여부 : 신제품\n판매시간 : 오전 10시 30분 ~ 새벽 4시\n영양소 정보:\n중량(g): 251g\n중량(ml): -\n열량: 621kcal\n당: 13g\n단백질: 29g\n포화지방: 12g\n나트륨: 985mg\n카페인: -\n원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)\n\n\n카테고리 : 버거\n메뉴 이름 : 맥스파이시 상하이 버거\n제품소개 : 쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에아삭아삭한 양상추와 신선한 토마토까지!더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로입맛도 기분도 화끈하게!\n메뉴 개시일 : 2024.06.05 00:00\n메뉴 마감일 : 2099.06.30 00:00\n판매시간 : 10:30AM~4AM\n영양소 정보:\n중량(g): 246g\n중량(ml):

In [5]:
from langchain.text_splitter import CharacterTextSplitter

# CharacterTextSplitter를 사용하여 문서 분할
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

Created a chunk of size 298, which is longer than the specified 100
Created a chunk of size 293, which is longer than the specified 100
Created a chunk of size 323, which is longer than the specified 100
Created a chunk of size 245, which is longer than the specified 100
Created a chunk of size 332, which is longer than the specified 100
Created a chunk of size 333, which is longer than the specified 100
Created a chunk of size 302, which is longer than the specified 100
Created a chunk of size 322, which is longer than the specified 100
Created a chunk of size 294, which is longer than the specified 100
Created a chunk of size 322, which is longer than the specified 100
Created a chunk of size 276, which is longer than the specified 100
Created a chunk of size 266, which is longer than the specified 100
Created a chunk of size 268, which is longer than the specified 100
Created a chunk of size 242, which is longer than the specified 100
Created a chunk of size 268, which is longer tha

In [6]:
print("분할된 문서 수:", len(split_docs))

분할된 문서 수: 138


### Embedding

In [7]:
from kiwipiepy import Kiwi
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [8]:
from langchain_upstage import UpstageEmbeddings

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

store = LocalFileStore("./.cache")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace="solar-embedding-1-large",  # Solar 임베딩 모델 이름으로 namespace 변경
)

faiss_vectorstore = FAISS.from_documents(
    split_docs,
    cached_embedder,
)

faiss = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

In [9]:
kiwi_bm25 = BM25Retriever.from_documents(split_docs, preprocess_func=kiwi_tokenize)
kiwi_bm25.k = 4

kiwibm25_faiss_37 = EnsembleRetriever(
    retrievers=[kiwi_bm25, faiss],  # 사용할 검색 모델의 리스트
    weights=[0.3, 0.7],  # 각 검색 모델의 결과에 적용할 가중치
    search_type="mmr",  # 검색 결과의 다양성을 증진시키는 MMR 방식을 사용
)

### LLM연결

**비용 비교 (Million token당)**
- GPT-4o　　　　　　　　: input 5, output 15
- GPT-4o 2024-08-06　 　: input 2.5, cached input token 1.25, output 10
- GPT-4o mini　　　　　  : input 0.15, cached input token 0.075, output 0.6
- gpt-3.5-turbo-0125 　　: input 0.5, output 1.5  
- Claude 3.5 sonnet 　　 : input 3, output 15

In [10]:
# # chatGPT
# from langchain_openai import ChatOpenAI
# from langchain.chains import RetrievalQA

# openai = ChatOpenAI(model_name="gpt-4o2024-08-06", temperature = 0)

# qa = RetrievalQA.from_chain_type(
#     llm = openai,
#     chain_type = "stuff",
#     retriever = kiwibm25_faiss_37,
#     return_source_documents = True
# )

In [11]:
# claude
from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA

anthropic = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")


qa = RetrievalQA.from_chain_type(
    llm = anthropic,
    chain_type = "stuff",
    retriever = kiwibm25_faiss_37,
    return_source_documents = True
)

In [12]:
def test(query):
    result = qa(query)
    print(f"LLM 답변: {result['result']}")
    print("="*50)

    for i in result['source_documents']:
        print(f"검색 결과 문서: {i.page_content}")
        print("-"*50)


## 테스트

In [13]:
# 검색 결과 문서를 가져옵니다.
query = "닭고기가 들어간 버거를 추천해줘"
test(query)

/tmp/ipykernel_11392/1499587858.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(query)


LLM 답변: 네, 닭고기가 들어간 버거를 몇 가지 추천해 드리겠습니다:

1. 맥스파이시 상하이 버거
   - 100% 닭가슴살 패티 사용
   - 쌀가루를 더해 바삭한 식감
   - 양상추와 토마토 포함

2. 맥크리스피 스리라차 마요
   - 100% 통닭다리살 케이준 패티 사용
   - 스리라차 마요 소스로 매콤하고 고소한 맛
   - 신제품

3. 맥치킨
   - 바삭한 치킨 패티 사용
   - 고소한 화이트 마요 소스와 아삭한 양상추 포함
   - 업그레이드된 맛

이 세 가지 버거 모두 닭고기를 주 재료로 사용하며, 각각 다른 특징과 맛을 가지고 있습니다. 개인의 취향에 따라 선택하시면 좋을 것 같습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥스파이시 상하이 버거
제품소개 : 쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에아삭아삭한 양상추와 신선한 토마토까지!더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로입맛도 기분도 화끈하게!
메뉴 개시일 : 2024.06.05 00:00
메뉴 마감일 : 2099.06.30 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 246g
중량(ml): -
열량: 501kcal
당: 9g
단백질: 24g
포화지방: 3g
나트륨: 1102mg
카페인: -
원산지 정보: 닭고기:태국산(경상 지역), 국내산(그 외 지역)
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥크리스피 스리라차 마요
제품소개 : 빠삭한 100% 통닭다리살 케이준 패티에스리라차 마요 소스를 더해 매콤 고소한 맛!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 289g
중량(ml): -
열량: 663kcal
당: 15g
단백질: 26g
포화지방: 7g
나트륨: 1508mg


In [14]:
query = "닭고기가 들어가지 않은 버거를 추천해줘"
test(query)

LLM 답변: 네, 닭고기가 들어가지 않은 버거를 추천해드리겠습니다:

1. 불고기 버거
   - 한국인의 입맛에 맞는 불고기 소스에 잘 재운 패티와 고소한 마요네즈, 신선한 양상추의 조합입니다.
   - 원산지: 돼지고기(국내산)

2. 1955 버거
   - 113g의 두툼한 순 쇠고기 패티, 풍성한 신선한 양상추, 그릴드 어니언이 들어간 오리지널 버거입니다.
   - 원산지: 쇠고기(호주산), 돼지고기(베이컨: 외국산 - 아일랜드, 스페인, 캐나다)

이 두 가지 버거는 닭고기 대신 돼지고기나 쇠고기를 사용하고 있어 닭고기 알레르기가 있거나 닭고기를 피하고 싶은 분들에게 좋은 선택이 될 수 있습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥스파이시 상하이 버거
제품소개 : 쌀가루가 더해져 더 바삭해진 100% 닭가슴살 패티에아삭아삭한 양상추와 신선한 토마토까지!더 바삭하고 맛있어진 NEW 맥스파이시 상하이 버거로입맛도 기분도 화끈하게!
메뉴 개시일 : 2024.06.05 00:00
메뉴 마감일 : 2099.06.30 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 246g
중량(ml): -
열량: 501kcal
당: 9g
단백질: 24g
포화지방: 3g
나트륨: 1102mg
카페인: -
원산지 정보: 닭고기:태국산(경상 지역), 국내산(그 외 지역)
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥크리스피 스리라차 마요
제품소개 : 빠삭한 100% 통닭다리살 케이준 패티에스리라차 마요 소스를 더해 매콤 고소한 맛!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 289g
중량(ml): -
열량: 663kcal
당: 15g
단백질: 26g
포화지방: 7g
나트륨: 1508mg
카페인: -


In [15]:
query = "치킨버거는 뭐가있어?"
test(query)

LLM 답변: 맥도날드의 치킨 버거 메뉴로는 다음과 같은 것들이 있습니다:

1. 맥치킨: 바삭한 치킨 패티와 고소한 화이트 마요 소스, 아삭한 양상추가 들어간 버거입니다.

2. 맥크리스피 클래식 버거: 100% 통닭다리살을 사용한 겉바삭 속촉한 케이준 치킨 패티로 만든 클래식한 치킨 버거입니다.

3. 맥크리스피 디럭스 버거: 100% 통닭다리살 케이준 치킨 패티에 촉촉한 포테이토 브리오쉬 번, 스페셜 스모키 소스가 더해진 프리미엄 치킨 버거입니다.

이 메뉴들은 단품으로도 주문 가능하고, 세트 메뉴로도 제공됩니다. 또한 맥크리스피 디럭스 버거는 맥런치 세트로도 제공되어 점심 시간에 특별 할인된 가격으로 즐길 수 있습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥치킨 세트
제품소개 : 바삭한 치킨 패티, 고소한 화이트 마요 소스와아삭한 양상추가 함께!더 업그레이드된 맛으로 돌아온 맥치킨
메뉴 개시일 : 2019.10.10 00:00
메뉴 마감일 : 9999.01.01 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): -
중량(ml): -
열량: 847~986kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보: 닭고기:태국산,국내산 섞음
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥치킨
제품소개 : 바삭한 치킨 패티, 고소한 화이트 마요 소스와아삭한 양상추가 함께!더 업그레이드된 맛으로 돌아온 맥치킨
메뉴 개시일 : 2019.10.10 00:00
메뉴 마감일 : 9999.01.01 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 206g
중량(ml): -
열량: 523kcal
당: 7g
단백질: 21g
포화지방: 3g
나트륨: 1084mg
카페인: -
원산지 정보: 닭고기:태국산,국내산 섞음
-----------------------------------------------

In [16]:
query = "신메뉴는 뭐가있어?"
test(query)

LLM 답변: 맥도날드의 최근 신메뉴로는 다음과 같은 제품들이 있습니다:

1. 베토디 스리라차 마요: 베이컨 토마토 디럭스에 스리라차 마요 소스를 더한 버거입니다.

2. 맥크리스피 스리라차 마요: 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더한 버거입니다.

3. 진도 대파 크림 크로켓 머핀: 100% 국내산 진도 대파가 들어간 크로켓을 사용한 맥모닝 메뉴입니다.

이 신메뉴들은 2024년 9월부터 10월 중순까지 (진도 대파 크림 크로켓 머핀은 11월 중순까지) 한정 판매되는 것으로 보입니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 베토디 스리라차 마요
제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 251g
중량(ml): -
열량: 621kcal
당: 13g
단백질: 29g
포화지방: 12g
나트륨: 985mg
카페인: -
원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 베토디 스리라차 마요 세트
제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): -
중량(ml): -
열량: 945~1085kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)
--------------------------------------------------
검색 결과 문서

In [27]:
query = "새로나온 버거는 뭐가있어?"
test(query)

LLM 답변: 제공된 정보에 따르면 최근에 새로 출시된 버거에 대한 구체적인 정보는 없습니다. 주어진 메뉴 목록은 대부분 2013년부터 2021년 사이에 출시된 버거들로, 가장 최근에 추가된 메뉴는 2021년 5월 19일에 출시된 '더블 불고기 버거 세트'입니다. 

그러나 이 정보가 최신 상태가 아닐 수 있으므로, 가장 최근의 새로운 버거에 대해 알고 싶으시다면 맥도날드의 공식 웹사이트나 앱을 확인하거나 매장에 직접 문의하시는 것이 좋을 것 같습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 슈비 버거
제품소개 : 탱~글한 통새우살에 비프 패티를 더해 푸짐하게~슈비 버거
메뉴 개시일 : 2016.10.07 00:00
메뉴 마감일 : 9999.11.04 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 269g
중량(ml): -
열량: 540kcal
당: 15g
단백질: 20g
포화지방: 5g
나트륨: 1001mg
카페인: -
원산지 정보: 새우패티:태국산쇠고기:호주산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 슈슈 버거
제품소개 : 탱~글한 통새우살 가득~ 슈슈 버거!
메뉴 개시일 : 2016.10.07 00:00
메뉴 마감일 : 9999.10.01 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 224g
중량(ml): -
열량: 409kcal
당: 13g
단백질: 12g
포화지방: 1.7g
나트륨: 808mg
카페인: -
원산지 정보: 새우패티:태국산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 슈슈 버거 세트
제품소개 : 탱~글한 통새우살 가득~ 슈슈 버거*
메뉴 개시일 : 2016.10.07 00:00
메뉴 마감일 : 9999.10.01 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 

In [18]:
query = "신제품 버거는 뭐가있어?"
test(query)

LLM 답변: 주어진 정보를 바탕으로 신제품으로 분류된 버거는 다음과 같습니다:

1. 베토디 스리라차 마요
   - 베이컨 토마토 디럭스에 스리라차 마요 소스를 더한 매콤한 버거

2. 토마토 치즈 비프 버거
   - 신선한 토마토와 고소한 치즈버거의 조화

3. 맥크리스피 디럭스 버거
   - 100% 통닭다리살을 사용한 케이준 치킨 패티와 스페셜 스모키 소스가 특징인 치킨 버거

이 세 가지 버거가 신제품으로 소개되어 있습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 트리플 치즈버거 세트
제품소개 : 부드러운 치즈와 풍부한 육즙의 패티를세배 더 진하게 즐길 수 있는 트리플 치즈버거
메뉴 개시일 : 2021.05.27 09:00
메뉴 마감일 : 9999.07.31 00:00
신제품 여부 : 신제품
판매시간 : 
영양소 정보:
중량(g): -
중량(ml): -
열량: 943~1083kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보:
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 베토디 스리라차 마요
제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 251g
중량(ml): -
열량: 621kcal
당: 13g
단백질: 29g
포화지방: 12g
나트륨: 985mg
카페인: -
원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 토마토 치즈 비프 버거 세트
제품소개 : 신선한 토마토와 고소한 치즈버거의 조화
메뉴 개시일 : 2024.06.13 00:00

In [19]:
query = "매콤한 메뉴는 뭐가있어?"
test(query)

LLM 답변: 매콤한 메뉴로는 다음과 같은 것들이 있습니다:

1. 베토디 스리라차 마요: 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함을 제공합니다.

2. 맥크리스피 스리라차 마요: 100% 통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤하고 고소한 맛을 냅니다.

3. 베이컨 토마토 디럭스 세트: 이 메뉴는 매콤달콤한 스위트 칠리 소스가 포함되어 있어 약간의 매운맛을 즐길 수 있습니다.

이 메뉴들은 모두 스리라차 마요 소스나 스위트 칠리 소스를 사용하여 매콤한 맛을 제공합니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 베토디 스리라차 마요
제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 251g
중량(ml): -
열량: 621kcal
당: 13g
단백질: 29g
포화지방: 12g
나트륨: 985mg
카페인: -
원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 맥크리스피 스리라차 마요
제품소개 : 빠삭한 100% 통닭다리살 케이준 패티에스리라차 마요 소스를 더해 매콤 고소한 맛!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 289g
중량(ml): -
열량: 663kcal
당: 15g
단백질: 26g
포화지방: 7g
나트륨: 1508mg
카페인: -
원산지 정보: 닭고기:태국산(경상지역), 브라질산(그 외 지역)
------------------------------------------------

In [20]:
query = "매콤하지 않은 메뉴는 뭐가있어?"
test(query)

LLM 답변: 제공된 정보에서 매콤하지 않은 메뉴로 확실하게 언급된 것은 다음과 같습니다:

1. 코울슬로: 양배추, 당근, 양파가 상큼하고 크리미한 마요 드레싱과 어우러진 아삭한 샐러드입니다.

2. 1955 버거: 순 쇠고기 패티, 양상추, 그릴드 어니언으로 구성된 오리지널 버거입니다.

이 두 메뉴는 제품 설명에서 매운맛에 대한 언급이 없어 매콤하지 않은 것으로 추정할 수 있습니다.

다른 메뉴들(맥스파이시 치킨 텐더, 베토디 스리라차 마요, 맥스파이시 상하이 버거)은 매운맛이나 스파이시한 특징이 언급되어 있어 매콤한 맛이 있는 것으로 보입니다.
검색 결과 문서: 카테고리 : 사이드 & 디저트
메뉴 이름 : 맥스파이시치킨 텐더
제품소개 : 부드러운 100% 닭안심살을 스파이시 시즈닝으로매콤 바삭하게 튀겨낸 치킨 텐더!2가지 소스로 입맛에 맞게 즐겨보세요!
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 
영양소 정보:
중량(g): -
중량(ml): -
열량: -
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보: 닭고기:국내산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 베토디 스리라차 마요
제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 251g
중량(ml): -
열량: 621kcal
당: 13g
단백질: 29g
포화지방: 12g
나트륨: 985mg
카페인: -
원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)
--------------------------------------------------
검색 결과 문서: 카테고리 : 사이드 

In [21]:
query = "매콤하지 않은 버거는 뭐가있어?"
test(query)

LLM 답변: 매콤하지 않은 버거로는 다음과 같은 옵션들이 있습니다:

1. 불고기 버거: 한국인의 입맛에 맞는 불고기 소스를 사용하며, 매콤하지 않습니다.

2. 더블 불고기 버거: 불고기 버거의 패티가 두 장인 버전으로, 역시 매콤하지 않습니다.

3. 1955 버거: 순 쇠고기 패티와 그릴드 어니언을 사용한 오리지널 버거로, 매콤한 맛은 없습니다.

이 버거들은 모두 매운 맛 대신 각각의 고유한 풍미를 가지고 있습니다. 불고기 버거는 달콤한 불고기 소스의 맛을, 1955 버거는 쇠고기와 그릴드 어니언의 깊은 맛을 즐길 수 있습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 불고기 버거
제품소개 : 한국인의 입맛에 딱 맞는 불고기 소스에잘 재운 패티와
고소한 마요네즈,신선한 양상추의 맛있는 조합.
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 167g
중량(ml): -
열량: 409kcal
당: 14g
단백질: 14g
포화지방: 6g
나트륨: 670mg
카페인: -
원산지 정보: 돼지고기:국내산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 불고기 버거 세트
제품소개 : 한국인의 입맛에 딱 맞는 불고기 소스에잘 재운 패티와 고소한 마요네즈,신선한 양상추의 맛있는 조합.
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): -
중량(ml): -
열량: 732~872kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보: 돼지고기:국내산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 더블 불고기 버거
제품소개 : 진한 불고

In [22]:
query = "가장 열량이 낮은 메뉴는 뭐가있어?"
test(query)

LLM 답변: 주어진 정보 중에서 가장 열량이 낮은 메뉴는 코카-콜라 제로 Medium입니다. 이 제품의 열량은 0kcal입니다.

다른 메뉴들의 열량 정보는 다음과 같습니다:

1. 코카-콜라 제로 Medium: 0kcal
2. 해쉬 브라운: 162kcal
3. 코울슬로: 179kcal
4. 베리 스트로베리 맥플러리: 325kcal
5. 쿼터파운더 치즈: 536kcal

나머지 메뉴들은 정확한 열량 정보가 제공되지 않았거나, 세트 메뉴로 열량 범위가 넓게 제시되어 있어 직접적인 비교가 어렵습니다.
검색 결과 문서: 카테고리 : 사이드 & 디저트
메뉴 이름 : 코울슬로
제품소개 : 양배추, 당근, 양파가 상큼하고 크리미한 마요 드레싱과어우러져 아삭하게 씹히는 샐러드
메뉴 개시일 : 2022.05.26 00:00
메뉴 마감일 : 2099.12.31 00:00
판매시간 : 
영양소 정보:
중량(g): 100g
중량(ml): -
열량: 179kcal
당: 18g
단백질: 1g
포화지방: 1g
나트륨: 307mg
카페인: -
원산지 정보:
--------------------------------------------------
검색 결과 문서: 카테고리 : 맥카페 & 음료
메뉴 이름 : 코카-콜라 제로
제품소개 : 상쾌한 맛은 살리면서 웰빙 트렌드에 맞춰설탕과 칼로리를 제로로 줄인 코카-콜라 제로
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 
영양소 정보:
중량(g): -
중량(ml): -
열량: -
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보:
--------------------------------------------------
검색 결과 문서: 카테고리 : 사이드 & 디저트
메뉴 이름 : 해쉬 브라운
제품소개 : 씹으면 바삭바삭,속은 부드러운 감자의 고소함.
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 새벽 

In [23]:
query = "가장 열량이 낮은 버거는 뭐가있어?"
test(query)

LLM 답변: 제공된 정보 중에서 가장 열량이 낮은 버거는 '햄버거'입니다. 햄버거의 열량은 266kcal로, 다른 버거들에 비해 가장 낮습니다.

비교를 위해 다른 버거들의 열량을 나열하면 다음과 같습니다:

1. 햄버거: 266kcal
2. 쿼터파운더 치즈: 536kcal
3. 1955 버거: 572kcal
4. 더블 쿼터파운더 치즈: 770kcal

따라서 열량을 가장 적게 섭취하고 싶다면 햄버거를 선택하는 것이 좋습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 1955 버거
제품소개 : 113g 두툼한 순 쇠고기 패티,특별한 1955 소스에 깊은 풍미의 그릴드 어니언까지!맥도날드가 처음 생긴 1955년의 맛을 담은영원한 오리지널 1955 버거
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 264g
중량(ml): -
열량: 572kcal
당: 15g
단백질: 29g
포화지방: 10g
나트륨: 930mg
카페인: -
원산지 정보: 쇠고기:호주산돼지고기(베이컨):외국산(아일랜드,스페인,캐나다)
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 햄버거
제품소개 : 100% 순 쇠고기 패티에 상큼한 피클과 양파,기본에 충실한 맥도날드 햄버거.
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 오전 10시 30분 ~ 새벽 4시
영양소 정보:
중량(g): 101g
중량(ml): -
열량: 266kcal
당: 7g
단백질: 13g
포화지방: 3g
나트륨: 450mg
카페인: -
원산지 정보: 쇠고기:호주산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 1955 버거 세트
제품소개 : 113g 두툼한 순 쇠고기 패티

In [24]:
query = "가장 양이 적은 버거는 뭐야?"
test(query)

LLM 답변: 주어진 정보를 바탕으로 가장 양이 적은 버거는 '햄버거'입니다. 

햄버거의 중량은 101g으로, 제시된 다른 버거들과 비교했을 때 가장 가벼운 중량을 가지고 있습니다. 또한 열량도 266kcal로 가장 낮습니다.

비교 대상:
- 더블 쿼터파운더 치즈: 275g, 770kcal
- 더블 불고기 버거: 237g, 635kcal
- 쿼터파운더 치즈: 198g, 536kcal
- 햄버거: 101g, 266kcal

따라서 제공된 정보 중에서 햄버거가 가장 양이 적은 버거라고 할 수 있습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 더블 쿼터파운더 치즈
제품소개 : 좋아하는건 더 많이 즐기시라고,두 배 이상 커진 1/4 파운드비프 패티가 두 장 (226그램)!육즙이 풍부한 고기 맛을그대로 살린 100% 순 쇠고기 패티 두 장과치즈 두 장이 입안 가득 완벽하게 조화되는 놀라운 맛
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 275g
중량(ml): -
열량: 770kcal
당: 10g
단백질: 50g
포화지방: 20g
나트륨: 1151mg
카페인: -
원산지 정보: 쇠고기:호주산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 더블 쿼터파운더 치즈 세트
제품소개 : 좋아하는건 더 많이 즐기시라고,두 배 이상 커진 1/4 파운드비프 패티가 두 장 (226그램)!육즙이 풍부한 고기 맛을그대로 살린 100% 순 쇠고기 패티 두 장과치즈 두 장이 입안 가득 완벽하게 조화되는 놀라운 맛
메뉴 개시일 : 2013.11.27 00:00
메뉴 마감일 : 9999.12.31 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): -
중량(ml): -
열량: 1094~1233kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산

In [25]:
query = "해산물이 들어간 버거는 뭐가있어?"
test(query)

LLM 답변: 해산물이 들어간 버거로는 다음과 같은 메뉴들이 있습니다:

1. 슈슈 버거: 통새우살이 들어간 버거입니다.
2. 슈비 버거: 통새우살과 비프 패티가 함께 들어간 버거입니다.

이 두 버거는 모두 태국산 새우패티를 사용하고 있습니다. 또한 각각의 세트 메뉴도 제공되고 있습니다.
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 슈슈 버거
제품소개 : 탱~글한 통새우살 가득~ 슈슈 버거!
메뉴 개시일 : 2016.10.07 00:00
메뉴 마감일 : 9999.10.01 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 224g
중량(ml): -
열량: 409kcal
당: 13g
단백질: 12g
포화지방: 1.7g
나트륨: 808mg
카페인: -
원산지 정보: 새우패티:태국산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 슈비 버거
제품소개 : 탱~글한 통새우살에 비프 패티를 더해 푸짐하게~슈비 버거
메뉴 개시일 : 2016.10.07 00:00
메뉴 마감일 : 9999.11.04 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): 269g
중량(ml): -
열량: 540kcal
당: 15g
단백질: 20g
포화지방: 5g
나트륨: 1001mg
카페인: -
원산지 정보: 새우패티:태국산쇠고기:호주산
--------------------------------------------------
검색 결과 문서: 카테고리 : 버거
메뉴 이름 : 슈슈 버거 세트
제품소개 : 탱~글한 통새우살 가득~ 슈슈 버거*
메뉴 개시일 : 2016.10.07 00:00
메뉴 마감일 : 9999.10.01 00:00
판매시간 : 10:30AM~4AM
영양소 정보:
중량(g): -
중량(ml): -
열량: 732~872kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보: 새우패티:태국산
--------------

In [26]:
query = "지금 시간이 오후 2시인데, 지금 시킬 수 있는 메뉴 중에 세트메뉴는 뭐가있어?"
test(query)

LLM 답변: 죄송합니다만, 오후 2시에는 맥런치 메뉴를 주문할 수 없습니다. 맥런치 세트는 오전 10시 30분부터 오후 2시까지만 판매됩니다.

하지만 일반 세트 메뉴 중에서는 다음 메뉴를 주문할 수 있습니다:

1. 쿼터파운더 치즈 세트

이 메뉴는 오전 10시 30분부터 오전 4시까지 판매되므로 오후 2시에도 주문 가능합니다.

다른 일반 세트 메뉴도 있을 수 있지만, 제공된 정보에는 그 외의 세트 메뉴에 대한 구체적인 판매 시간이 명시되어 있지 않습니다. 더 많은 선택지를 알고 싶으시다면 매장에 직접 문의하시는 것이 좋을 것 같습니다.
검색 결과 문서: 카테고리 : 맥런치
메뉴 이름 : 더블 불고기 버거 세트
제품소개 : 진한 불고기 소스의 패티가 두 장!여기에 고소한 치즈와 마요네즈,신선한 양상추를 곁들인 깊고 풍부한 맛.점심만의 특별한 할인으로 즐길 수 있는 [맥런치 세트]를 만나보세요* 맥런치 * 맥런치 할인은 매장 방문에 한함
메뉴 개시일 : 2021.05.19 19:00
메뉴 마감일 : 9999.05.19 00:00
판매시간 : 오전 10시 30분부터 오후 2시까지
영양소 정보:
중량(g): -
중량(ml): -
열량: 959~1098kcal
당: -
단백질: -
포화지방: -
나트륨: -
카페인: -
원산지 정보: 돼지고기:국내산
--------------------------------------------------
검색 결과 문서: 카테고리 : 맥런치
메뉴 이름 : 베토디 스리라차 마요 세트
제품소개 : 베이컨 토마토 디럭스에스리라차 마요 소스를 더해 색다른 매콤함!점심만의 특별한 할인으로 즐길 수 있는 [맥런치 세트]를 만나보세요* 맥런치 * 맥런치 할인은 매장 방문에 한함
메뉴 개시일 : 2024.09.13 00:00
메뉴 마감일 : 2024.10.16 23:59
신제품 여부 : 신제품
판매시간 : 오전 10시 30분부터 오후 2시까지
영양소 정보:
중량(g): -
중량(ml): -
열량: 945~1085kcal
당: -
단백질: -